In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
print(LANGSMITH_API_KEY)

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


In [3]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY

In [4]:
from langchain_groq import ChatGroq
llm = ChatGroq(model = "llama-3.3-70b-versatile")

In [5]:
while True:
    question  = input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

goodbye take care yourself


In [6]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [7]:
store = {}

In [8]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [9]:
config = {"configurable": {"session_id": "firstchat"}}


In [10]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)


In [11]:
model_with_memory.invoke(('Hi I am Vibhav Ahuja'), config=config).content

"Hello Vibhav Ahuja, it's nice to meet you. Is there something I can help you with or would you like to chat?"

In [12]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi I am Vibhav Ahuja', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Vibhav Ahuja, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 43, 'total_tokens': 73, 'completion_time': 0.109090909, 'prompt_time': 0.001944977, 'queue_time': 0.054855403, 'total_time': 0.111035886}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'finish_reason': 'stop', 'logprobs': None}, id='run--3594b7c4-2d38-4a0f-8e1b-f3de6bf15ee9-0', usage_metadata={'input_tokens': 43, 'output_tokens': 30, 'total_tokens': 73})])}

In [13]:
model_with_memory.invoke(('what is my name'), config=config).content

'Your name is Vibhav Ahuja.'

In [14]:
# from sentence_transformers import SentenceTransformer
# embedding = SentenceTransformer("BAAI/bge-m3")
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})  


d:\Ai Agents\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [16]:
loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

In [17]:
print(docs)

[Document(metadata={'source': '..\\data\\Generic Modular RAG Documentation.txt'}, page_content="ï»¿Generic Modular RAG Documentation\n\n\nOverview\nGeneric Modular RAG (Retrieval-Augmented Generation) is a flexible and adaptable framework designed for efficient information retrieval and generation across various data formats and domains. It provides a plug-and-play architecture where individual components such as document loaders, chunkers, retrievers, vector stores, LLMs, and re-rankers can be integrated and customized as needed.\nKey features:\n* Modular design for easy customization and extension.\n* Compatibility with diverse file types and formats.\n* Scalable for both small-scale and enterprise-level applications.\n* Enables explainable and traceable outputs suitable for production environments.\nStep 1: Data Preprocessing & File Extraction\nTo begin with, preprocess and extract content from various file types including .txt, .pdf, .csv, .xlsx, and .docx. Below are the recommende

In [18]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
)
new_docs = text_splitter.split_documents(documents=docs)


In [19]:
db = FAISS.from_documents(new_docs, embedding)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [20]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [21]:
retrieval_chain = (
    RunnableParallel({'context': retriever, 'question': RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
question = "what is gen ai? give me 3 points"
print(retrieval_chain.invoke(question))

Based on the provided context, here are 3 points about Generation (referred to as "Gen AI"):

1. **Flexible framework**: Generation is described as a "flexible and adaptable framework".
2. **Part of a larger architecture**: It is mentioned in the context of a "plug-and-play architecture", suggesting that Generation is a component of a larger system.
3. **Related to Modular RAG**: The mentions of Generation and other concepts (e.g., Milvus, Overview) are all sourced from the "Generic Modular RAG Documentation.txt", implying a connection between Generation and Modular RAG.


In [23]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [24]:
api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=300)

In [25]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [26]:
print(tool.name)
print(tool.description)
print(tool.args)

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
{'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}


In [27]:
print(tool.run({'query':'langchain'}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis a


In [28]:
tool.run('langchain')

"Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis a"

In [29]:
from langchain_core.pydantic_v1 import BaseModel, Field

d:\Ai Agents\env\Lib\site-packages\IPython\core\interactiveshell.py:3672: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
class WikiInputs(BaseModel):
    query : str = Field(description="query to look up in Wikipedia, should be 3 or less words")

In [ ]:
tool = WikipediaQueryRun(
    name = 'wiki-tool',
    description='look up things in wikipedia',
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True
)

In [31]:
from langchain import hub

In [32]:
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults


In [33]:
tool = TavilySearchResults()


C:\Users\vibhav.ahuja\AppData\Local\Temp\ipykernel_18080\4050003858.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults()


In [34]:
tool.invoke({"query": "What happened in the latest burning man floods"})


[{'title': 'The latest on the Burning Man flooding - NPR',
  'url': 'https://www.npr.org/2023/09/03/1197497458/the-latest-on-the-burning-man-flooding',
  'content': "Knee-deep mud, warnings to conserve food and water, orders to shelter in place - this is all at Burning Man 2023 after torrential rains turned the Black Rock Desert into miles and miles of mud. There are also reports that at least one person has died at the counterculture festival about a hundred miles north of Reno, Nev. Earlier this afternoon, I caught up with NPR's Claudia Peschiutta, who's at her first burn, and she told me it's muddy where she is, but that she and her camp family have been [...] Knee-deep mud, warnings to conserve food and water, orders to shelter in place - this is all at Burning Man 2023 after torrential rains turned the Black Rock Desert into miles and miles of mud. There are also reports that at least one person has died at the counterculture festival about a hundred miles north of Reno, Nev. Earl

In [35]:
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [36]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] optional_variables=['chat_history'] input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')]

In [37]:
tavily_tool = TavilySearchResults()

In [38]:
tools = [tavily_tool]

In [39]:
print(tools)

[TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))]


In [45]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [46]:
agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    verbose = True
)

In [49]:
print(agent_executor.invoke({"input": "explain about india at least 10 points and give answer point wise"}))




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'India'}`
responded: Here are 10 points about India:



[{'title': 'India - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/India', 'content': '**India**, officially the **Republic of India**,[[j]](https://en.wikipedia.org/wiki/India#cite_note-29)[[20]](https://en.wikipedia.org/wiki/India#cite_note-30) is a country in [South Asia](https://en.wikipedia.org/wiki/South_Asia "South Asia"). It is the [seventh-largest country by area](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area "List of countries and dependencies by area"); the [most populous [...] India has been a [federal republic](https://en.wikipedia.org/wiki/Federal_republic "Federal republic") since 1950, governed through a democratic [parliamentary system](https://en.wikipedia.org/wiki/Parliamentary_system "Parliamentary system"). It is a [pluralistic](https://en.wikipedia.org/wiki/Pluralism_(political_philoso

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor,  create_tool_calling_agent
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults

llm = ChatGroq(model="llama3-70b-8192") 

tavily_tool = TavilySearchResults()
tools = [tavily_tool]

print("Tool test:", tavily_tool.invoke({"query": "explain about india"}))

instructions = "You are an assistant."
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": "explain about india"})
print("Final response:", response)


Tool test: [{'title': 'India | History, Maps, Population, Economy, & Facts | Britannica', 'url': 'https://www.britannica.com/place/India', 'content': '**India**, [country](https://www.britannica.com/topic/nation-state) that occupies the greater part of [South Asia](https://www.britannica.com/place/South-Asia). It is made up of 28 states and eight [union territories](https://www.britannica.com/topic/union-territory), and its national capital is [New Delhi](https://www.britannica.com/place/New-Delhi), built in the 20th century just south of the historic hub of Old Delhi to serve as India’s administrative center. Its government is a [...] India remains one of the most ethnically diverse countries in the world. Apart from its many religions and sects, India is home to innumerable castes and tribes, as well as to more than a dozen major and hundreds of minor linguistic groups from several language families unrelated to one another. Religious minorities, including Muslims, Christians, Sikhs,